In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd
from bs4 import BeautifulSoup
import re
import sqlite3

### Scraping Cuenta de Resultados

In [83]:
# Función para limpiar y convertir valores a numéricos
def convertir_a_numerico(valor):
  if isinstance(valor, str):
    # Extraer números, puntos y comas
    valor = re.sub(r'[^\d.,-]', '', valor)
    # Reemplazar comas por puntos si es necesario
    valor = valor.replace(',', '.')
    try:
      return float(valor)
    except ValueError:
      return None
  return valor

def scrap_resultados(tk):
    dfs = []
    driver = webdriver.Chrome()
    try:
        # Abrir la URL
        url = f"https://es.investing.com/equities/{tk}-income-statement"
        driver.get(url)

        # Esperar a que al menos una tabla esté presente
        
        wait = WebDriverWait(driver, 5)
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "table")))

        # Obtener el HTML de la página
        page_html = driver.page_source

        # Parsear el HTML de la página con BeautifulSoup
        soup = BeautifulSoup(page_html, "html.parser")

        # Encontrar todas las tablas en la página
        tables = soup.find_all("table")

        # Procesar cada tabla
        for i, table in enumerate(tables):
            # Extraer las filas de la tabla
            rows = table.find_all("tr")
            data = []
            for row in rows:
                cells = row.find_all(["td", "th"])  # Buscar celdas (td) y encabezados (th)
                cell_texts = [cell.get_text(strip=True) for cell in cells]
                data.append(cell_texts)

            # Convertir los datos en un DataFrame de Pandas
            df = pd.DataFrame(data)
            dfs.append(df)
        print("exito al procesar!")

    except TimeoutException:
        print("Error: No se encontraron tablas en la página.")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        # Cerrar el navegador
        driver.quit()
        time.sleep(5)
    return dfs
  
def procesar_tab(df):
  data = pd.DataFrame()
   
  data = df[1].replace("aa.aa",None).reset_index(drop=True)
  data = data.drop(data[data[1]==""].index,axis=0)
  data = data[data[1].notna()]
  data = data.drop(columns=[0])
  data.columns = data.iloc[0]
  data.drop(0,inplace=True,axis=0)
  data = data.dropna(axis=1,how="all")
  data.rename(columns={data.columns[0]:"indicador"},inplace=True)


  # Aplicar la función a todas las columnas excepto 'company'
  ejercicios = [col for col in data.columns if col.startswith("20")]
  for col in ejercicios:  
    data[col] = data[col].apply(convertir_a_numerico)
  
  data.rename(columns=lambda x: x[:4] if x.startswith('20') else x, inplace=True)

  return data.set_index("indicador")

def get_resultados(tk):
  dfs = scrap_resultados(tk)
  if len(dfs) > 0:
    df = procesar_tab(dfs)
    df["company"] = tk
    df["informe"] = "resultados"
    return df
  else:
    return None
  

### Scraping hoja de Balance (situacion patrimonial)

In [57]:

def scrap_balance_sheet(tk):
    dfs = []
    # Configurar el navegador
    # chrome_options = Options()
    # chrome_options.add_argument("--headless")
    driver = webdriver.Chrome()
    try:
        # Abrir la URL
        url = f"https://es.investing.com/equities/{tk}-balance-sheet"
        driver.get(url)

        # Esperar a que al menos una tabla esté presente
        
        wait = WebDriverWait(driver, 5)
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "table")))

        # Obtener el HTML de la página
        page_html = driver.page_source

        # Parsear el HTML de la página con BeautifulSoup
        soup = BeautifulSoup(page_html, "html.parser")

        # Encontrar todas las tablas en la página
        tables = soup.find_all("table")

        # Procesar cada tabla
        for i, table in enumerate(tables):
            # Extraer las filas de la tabla
            rows = table.find_all("tr")
            data = []
            for row in rows:
                cells = row.find_all(["td", "th"])  # Buscar celdas (td) y encabezados (th)
                cell_texts = [cell.get_text(strip=True) for cell in cells]
                data.append(cell_texts)

            # Convertir los datos en un DataFrame de Pandas
            df = pd.DataFrame(data)
            dfs.append(df)
        print("exito al procesar!")

    except TimeoutException:
        print("Error: No se encontraron tablas en la página.")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        # Cerrar el navegador
        driver.quit()
        time.sleep(5)
    return dfs
  
def procesar_tab(df):
  titulos = df[1].T[0].dropna().reset_index(drop=True).tolist()[1:]
  titulos[0] = "indicador"

  result = pd.DataFrame()
   
  estructura = [
  {"seccion":"activo_corriente", "rubros":2,"valores":3},
  {"seccion":"activo_total", "rubros":4,"valores":5},
  {"seccion":"pasivo_corriente", "rubros":6,"valores":7},
  {"seccion":"pasivo_total", "rubros":8,"valores":9},
  {"seccion":"patrimonio_neto", "rubros":10,"valores":11},
  {"seccion":"crecimiento_deuda", "rubros":14,"valores":15},
]
  for e in estructura:
    try:
      data =  pd.concat([df[e["rubros"]][1],df[e["valores"]]], axis=1)
      data.columns = titulos
      data.replace("aa.aa",None,inplace=True)
      data.dropna(inplace=True, axis=1)
      data["seccion"] = e["seccion"]
      
      result = pd.concat([result, data.reset_index(drop=True)], axis=0)
    except:
      pass
  
    # Aplicar la función a todas las columnas excepto 'company'
  ejercicios = [col for col in result.columns if col.startswith("20")]
  for col in ejercicios:  # se aplica a las columnas de ejercicios
    result[col] = result[col].apply(convertir_a_numerico)
  
  result.rename(columns=lambda x: x[:4] if x.startswith('20') else x, inplace=True)
  
  return result.set_index("indicador")

def get_balance_sheet(tk):
  dfs = scrap_balance_sheet(tk)
  if len(dfs) > 0:
    df = procesar_tab(dfs)
    df["company"] = tk
    df["informe"] = "balance"
    return df
  else:
    return None

### Guardar datos anuales en bbdd

In [53]:
def save_datos_anuales(df):
    # Conectar a la base de datos (o crearla si no existe)
    conn = sqlite3.connect('webmining.db')
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS datos_anuales (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        company TEXT NOT NULL,
        informe TEXT NOT NULL,
        ejercicio INTEGER NOT NULL,
        indicador TEXT NOT NULL,
        valor REAL
    )
    ''')
    # Insertar datos
    ejercicios = [col for col in df.columns if col.isnumeric()]
    for ejercicio in ejercicios:
        # eliminar los datos para evitar duplicados
        company = df['company'][0]
        informe = df['informe'][0]
        cursor.execute(f"""
                    DELETE FROM datos_anuales 
                    WHERE company = '{company}' AND 
                    informe = '{informe}' AND 
                    ejercicio = {ejercicio}
                    """)
        conn.commit()
        
        # Estructurar e insertar los datos
        df_ej = df[[ejercicio, "company", "informe"]].copy().reset_index()
        df_ej["ejercicio"] = int(ejercicio)
        df_ej.rename(columns={ejercicio: "valor"}, inplace=True)
        df_ej.dropna(subset=["valor"], inplace=True)
        df_ej.to_sql('datos_anuales', conn, if_exists='append', index=False)
        # Guardar los cambios
        conn.commit()
    # Cerrar la conexión
    conn.close()

In [86]:
companies = [
  "apple-computer-inc",
  "microsoft-corp",
  "google-inc",
  "tesla-motors",
  "visa-inc",
  "berkshire-hathaway-inc",
  "johnson-johnson",
  "pfizer",
  "amazon-com-inc",
  "disney",
  "nike",
  "procter-gamble",
  "coca-cola-co",
  "chevron",
  "3m-co"
]

In [87]:
for company in companies:
    print(f"Procesando {company}...")
    try:
        # Procesar los datos
        df_resultados = get_resultados(company)
        if df_resultados is not None:
            save_datos_anuales(df_resultados)
        # df_balance = get_balance_sheet(company)
        # if df_balance is not None:
        #     save_datos_anuales(df_balance)
        print(f"{company} procesado con éxito!")
    except Exception as e:
        print(f"Error al procesar {company}: {e}")
        

Procesando apple-computer-inc...
exito al procesar!


C:\Users\carre\AppData\Local\Temp\ipykernel_29244\3596493551.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company = df['company'][0]
C:\Users\carre\AppData\Local\Temp\ipykernel_29244\3596493551.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  informe = df['informe'][0]
C:\Users\carre\AppData\Local\Temp\ipykernel_29244\3596493551.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company = df['company'][0]
C:\Users\carr

apple-computer-inc procesado con éxito!
Procesando microsoft-corp...
exito al procesar!


C:\Users\carre\AppData\Local\Temp\ipykernel_29244\3596493551.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company = df['company'][0]
C:\Users\carre\AppData\Local\Temp\ipykernel_29244\3596493551.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  informe = df['informe'][0]
C:\Users\carre\AppData\Local\Temp\ipykernel_29244\3596493551.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company = df['company'][0]
C:\Users\carr

microsoft-corp procesado con éxito!
Procesando google-inc...
exito al procesar!


...

C:\Users\carre\AppData\Local\Temp\ipykernel_29244\3596493551.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company = df['company'][0]
C:\Users\carre\AppData\Local\Temp\ipykernel_29244\3596493551.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  informe = df['informe'][0]
C:\Users\carre\AppData\Local\Temp\ipykernel_29244\3596493551.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company = df['company'][0]
C:\Users\carr

chevron procesado con éxito!
Procesando 3m-co...
exito al procesar!


C:\Users\carre\AppData\Local\Temp\ipykernel_29244\3596493551.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company = df['company'][0]
C:\Users\carre\AppData\Local\Temp\ipykernel_29244\3596493551.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  informe = df['informe'][0]
C:\Users\carre\AppData\Local\Temp\ipykernel_29244\3596493551.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company = df['company'][0]
C:\Users\carr

3m-co procesado con éxito!


C:\Users\carre\AppData\Local\Temp\ipykernel_29244\3596493551.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  company = df['company'][0]
C:\Users\carre\AppData\Local\Temp\ipykernel_29244\3596493551.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  informe = df['informe'][0]


In [64]:

# Consultar datos
conn = sqlite3.connect('webmining.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM datos_anuales WHERE company = '3m-co' AND informe = 'resultados'")
filas = cursor.fetchall()

for fila in filas:
    print(fila)

(3971, '3m-co', 'resultados', 2020, 'Crecmiento total de otros gastos operativos', -2.71)
(3972, '3m-co', 'resultados', 2020, 'Gastos de I+D', 1.862)
(3973, '3m-co', 'resultados', 2020, 'Gastos de venta, generales y administrativos', 6.667)
(3974, '3m-co', 'resultados', 2021, 'Crecmiento total de otros gastos operativos', 3.25)
(3975, '3m-co', 'resultados', 2021, 'Gastos de I+D', 1.994)
(3976, '3m-co', 'resultados', 2021, 'Gastos de venta, generales y administrativos', 6.812)
(3977, '3m-co', 'resultados', 2022, 'Crecmiento total de otros gastos operativos', -20.96)
(3978, '3m-co', 'resultados', 2022, 'Gastos de I+D', 1.16)
(3979, '3m-co', 'resultados', 2022, 'Gastos de venta, generales y administrativos', 5.8)
(3980, '3m-co', 'resultados', 2023, 'Crecmiento total de otros gastos operativos', -21.91)
(3981, '3m-co', 'resultados', 2023, 'Gastos de I+D', 1.123)
(3982, '3m-co', 'resultados', 2023, 'Gastos de venta, generales y administrativos', 4.312)
(3983, '3m-co', 'resultados', 2024, 'C

In [ ]:
conn = sqlite3.connect('webmining.db')
cursor = conn.cursor()
cursor.execute("SELECT COUNT* FROM datos_anuales")
numero_registros = cursor.fetchone()[0]
numero_registros

423

In [85]:
dt = get_resultados("3m-co")
dt

exito al procesar!


,2020,2021,2022,2023,2024,company,informe
indicador,,,,,,,
Ingresos totales,32.184,35.355,26.161,24.610,24.575,3m-co,resultados
Crecimiento de los ingresos totales,0.150,9.850,-26.000,-5.930,-0.140,3m-co,resultados
Coste de los ingresos,16.499,18.795,15.853,14.894,14.417,3m-co,resultados
Beneficio bruto,15.685,16.560,10.308,9.716,10.158,3m-co,resultados
Crecimiento del beneficio bruto,4.070,5.580,-37.750,-5.740,4.550,3m-co,resultados
Margen de beneficio bruto %,48.740,46.840,39.400,39.480,41.330,3m-co,resultados
Total de otros gastos operativos,8.529,8.806,6.960,5.435,5.868,3m-co,resultados
Crecmiento total de otros gastos operativos,NaN,NaN,NaN,NaN,NaN,3m-co,resultados
Gastos de I+D,NaN,NaN,NaN,NaN,NaN,3m-co,resultados


In [82]:
dt

,2020,2021,2022,2023,2024,seccion,company,informe
indicador,,,,,,,,
Crecmiento total de otros gastos operativos,-2.710,3.250,-20.96,-21.910,7.970,crecimiento_deuda,3m-co,resultados
Gastos de I+D,1.862,1.994,1.16,1.123,1.070,crecimiento_deuda,3m-co,resultados
"Gastos de venta, generales y administrativos",6.667,6.812,5.80,4.312,4.798,crecimiento_deuda,3m-co,resultados
Otros gastos operativos,NaN,NaN,NaN,NaN,NaN,crecimiento_deuda,3m-co,resultados


In [66]:
df = scrap_resultados("3m-co")
df

exito al procesar!


[   0        1     2             3     4       5            6
 0        None  None          None  None    None         None
 1     Símbolo               Bolsa        Divisa         None
 2         MMM          Nueva York           USD  Tiempo real
 3         MMM              México           MXN     Demorada
 4        1MMM               Milán           EUR  Tiempo real
 5         MMM               Suiza           CHF     Demorada
 6         MMM           Fráncfort           EUR     Demorada
 7         MMM               Xetra           EUR     Demorada
 8         MMM           TradeGate           EUR     Demorada
 9        MMMm        Buenos Aires           ARS     Demorada
 10     MMMC34                  B3           BRL     Demorada
 11        MMM               Viena           EUR  Tiempo real,
    0                                    1          2          3          4   \
 0                       Período terminado:  201531/12  201631/12  201731/12   
 1                         Ingres

In [84]:
dt = procesar_tab(df)
dt

,2020,2021,2022,2023,2024
indicador,,,,,
Ingresos totales,32.184,35.355,26.161,24.610,24.575
Crecimiento de los ingresos totales,0.150,9.850,-26.000,-5.930,-0.140
Coste de los ingresos,16.499,18.795,15.853,14.894,14.417
Beneficio bruto,15.685,16.560,10.308,9.716,10.158
Crecimiento del beneficio bruto,4.070,5.580,-37.750,-5.740,4.550
Margen de beneficio bruto %,48.740,46.840,39.400,39.480,41.330
Total de otros gastos operativos,8.529,8.806,6.960,5.435,5.868
Crecmiento total de otros gastos operativos,NaN,NaN,NaN,NaN,NaN
Gastos de I+D,NaN,NaN,NaN,NaN,NaN


In [80]:
data = df[1].replace("aa.aa",None).reset_index(drop=True)
data = data.drop(data[data[1]==""].index,axis=0)
data = data[data[1].notna()]
data = data.drop(columns=[0])
data.columns = data.iloc[0]
data.drop(0,inplace=True,axis=0)
data = data.dropna(axis=1,how="all")

# Aplicar la función a todas las columnas excepto 'company'
ejercicios = [col for col in data.columns if col.startswith("20")]
for col in ejercicios:  
  data[col] = data[col].apply(convertir_a_numerico)

data.rename(columns=lambda x: x[:4] if x.startswith('20') else x, inplace=True)

data

,Período terminado:,2020,2021,2022,2023,2024
1,Ingresos totales,32.184,35.355,26.161,24.610,24.575
3,Crecimiento de los ingresos totales,0.150,9.850,-26.000,-5.930,-0.140
5,Coste de los ingresos,16.499,18.795,15.853,14.894,14.417
7,Beneficio bruto,15.685,16.560,10.308,9.716,10.158
9,Crecimiento del beneficio bruto,4.070,5.580,-37.750,-5.740,4.550
11,Margen de beneficio bruto %,48.740,46.840,39.400,39.480,41.330
13,Total de otros gastos operativos,8.529,8.806,6.960,5.435,5.868
15,Crecmiento total de otros gastos operativos,NaN,NaN,NaN,NaN,NaN
16,Gastos de I+D,NaN,NaN,NaN,NaN,NaN
17,"Gastos de venta, generales y administrativos",NaN,NaN,NaN,NaN,NaN
